<a href="https://colab.research.google.com/github/ITorresz/Agentes/blob/main/Agente_que_busca_informa%C3%A7%C3%B5es_no_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
%pip -q install google-genai

In [51]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [52]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [53]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents="Quando é a próxima Imersão IA com google gemini da alura?"
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn, Twitter e Facebook da Alura. Eles costumam anunciar as novas edições das Imersões por lá.
*   **Verificar a página de Imersões da Alura:** Acesse a página de Imersões no site da Alura e veja se há alguma Imersão IA com Google Gemini programada: [https://www.alura.com.br/imersao](https://www.alura.com.br/imersao)
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre os próximos cursos, eventos e Imersões.
*   **Entrar no servidor do Discord da comunidade da Alura:** A comunidade da Alura no Discord pode ter informações sobre as próximas edições das Imersões.

In [54]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio. O curso é dividido em cinco aulas online e ensina a desenvolver projetos no Gemini, o assistente de IA do Google. O curso é gratuito e oferece certificado.

In [55]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini data']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [56]:
!pip install -q google-adk


In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [57]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [58]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [59]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction="""
        Vocé é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google(google_search)
        para recuperar as ultímas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no maxímo 5 lançamentos relevantes, com base na quantidade e entusiasmo das noticias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possívelo que ele não seja tão relevante assim
        e pode ser substítuido por outro tema mais.
        Esses lançamentos relevantes deve ser atuais, de no maxímo um mês antes da data de hoje.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [60]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes socias. Com base na lista de
        lamçamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderiamos abordar em um post sobre
        cada um deles. Voce tambçém pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        eretornar esse tema, seus pontos mais relevantes, e um plano com ois assusntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [78]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [79]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [81]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: futebol
Maravilha! Vamos então criar o post sobre novidades em futebol

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas buscas que podem me ajudar a encontrar os lançamentos mais relevantes no futebol no último mês:
> 
> 
> Com base nas notícias e tendências atuais no mundo do futebol, aqui estão alguns lançamentos e acontecimentos de destaque que provavelmente serão relevantes em maio de 2025:
> 
> **1. Competições de Clubes:**
> 
> *   **Novo Mundial de Clubes da FIFA:** A estreia do novo formato do Mundial de Clubes, com 32 equipes, está prevista para junho de 2025 nos Estados Unidos. Este evento é aguardado com grande expectativa e promete ser um dos maiores lançamentos do ano no futebol. O Los Angeles FC e o América disputarão um playoff em 31 de maio para definir a última vaga na fase de grupos. O vencedor entrará no Grupo D, que já conta com Flamengo, Espérance de Tunis e Chelsea.
> *   **Finais das Competições Europeias:** As finais da UEFA Champions League, Europa League e Conference League acontecerão em maio de 2025. As cidades de Munique, Bilbau e Wroclaw sediarão esses eventos, respectivamente.
> 
> **2. Ligas Nacionais e Regionais:**
> 
> *   **Competições Europeias:** As principais ligas europeias, como a Premier League, La Liga e Serie A Italiana, estão em andamento e se estenderão até maio de 2025. As finais da Copa do Rei (Espanha), Copa da Itália e Copa da Inglaterra também serão decididas entre abril e maio de 2025.
> *   **Brasileirão Série A:** O Campeonato Brasileiro Série A estará em andamento em maio, com clássicos e jogos importantes.
> *   **Mercado de Transferências:** A janela de transferências de verão no futebol brasileiro se aproxima, com clubes buscando reforços para o Mundial de Clubes e para a sequência da temporada. Rumores e negociações envolvendo jogadores como Lima e Jorginho estão circulando.
> *   **Taça Revelação:** O Benfica Sub-23 se classificou para as semifinais da Taça Revelação após vencer o Estrela da Amadora por 7-0.
> 
> **3. Jogadores em Destaque:**
> 
> *   **Jovens Talentos:** Estêvão, do Palmeiras, é apontado como um dos jovens jogadores a serem observados em 2025. Outros jovens talentos incluem Fidel Ambriz (Monterrey) e Jamie Bynoe-Gittens (Borussia Dortmund).
> *   **NXGN 2025:** A Goal.com listou os 50 melhores jovens talentos do futebol mundial nascidos a partir de janeiro de 2006. Chido Obi, do Manchester United, está entre os destaques.
> *   **Promessas para 2025:** Gabriel Silva (Sporting), Lucas Taibo (Sporting) e Gustavo Varela (Benfica) são apontados como jovens jogadores com potencial para se destacarem em 2025.
> 
> **4. Transferências:**
> 
> *   Pietro Comuzzo renovou com a Fiorentina até 2029.
> *   Álvaro Carreras pode se transferir para o Real Madrid.
> *   Tiago Fernandes deixará o comando técnico do Torreense.
> 
> Esta lista destaca alguns dos principais lançamentos e tendências no mundo do futebol em maio de 2025. As notícias estão em constante evolução, então é bom ficar atento para mais atualizações.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos e acontecimentos de destaque no mundo do futebol em maio de 2025, posso sugerir os seguintes pontos para posts em redes sociais:
> 
> **1. Novo Mundial de Clubes da FIFA:**
> 
> *   **Pontos Relevantes:**
>     *   Estreia do novo formato com 32 equipes.
>     *   Local: Estados Unidos, em junho de 2025.
>     *   Playoff entre Los Angeles FC e América para a última vaga.
>     *   Presença de grandes clubes como Flamengo, Chelsea e Espérance de Tunis.
> *   **Assuntos para o Post:**
>     *   Explicar o novo formato do Mundial de Clubes e sua importância.
>     *   Analisar os grupos e os principais confrontos esperados.
>     *   Destacar a preparação dos clubes e as expectativas para o torneio.
> 
> **2. Finais das Competições Europeias:**
> 
> *   **Pontos Relevantes:**
>     *   Finais da Champions League, Europa League e Conference League em maio de 2025.
>     *   Cidades-sede: Munique, Bilbau e Wroclaw.
>     *   Disputa entre os melhores clubes da Europa pelos títulos.
> *   **Assuntos para o Post:**
>     *   Analisar os finalistas e seus caminhos até a final.
>     *   Destacar os jogadores-chave e as táticas esperadas para os jogos.
>     *   Criar enquetes e promover discussões sobre os favoritos.
> 
> **3. Jovens Talentos:**
> 
> *   **Pontos Relevantes:**
>     *   Destaque para Estêvão (Palmeiras), Fidel Ambriz (Monterrey) e Jamie Bynoe-Gittens (Borussia Dortmund).
>     *   Lista NXGN 2025 com os 50 melhores jovens talentos nascidos a partir de 2006.
>     *   Promessas como Gabriel Silva (Sporting), Lucas Taibo (Sporting) e Gustavo Varela (Benfica).
> *   **Assuntos para o Post:**
>     *   Apresentar os principais jovens talentos e suas características.
>     *   Mostrar vídeos e lances dos jogadores em destaque.
>     *   Debater sobre o futuro desses jovens no futebol mundial.
> 
> **Escolha do Tema Mais Relevante e Plano Detalhado:**
> 
> Considerando o impacto global e o interesse do público, o tema mais relevante é o **Novo Mundial de Clubes da FIFA**.
> 
> **Plano Detalhado para um Post sobre o Mundial de Clubes:**
> 
> *   **Assunto 1: O que é o Novo Mundial de Clubes?**
>     *   Explicar o novo formato com 32 equipes, substituindo o antigo modelo.
>     *   Detalhar o período e o local de realização: junho de 2025, nos Estados Unidos.
>     *   Mencionar o objetivo da FIFA em tornar o torneio mais competitivo e global.
> *   **Assunto 2: Clubes Participantes e Expectativas**
>     *   Listar alguns dos principais clubes já classificados, como Flamengo, Chelsea e Real Madrid.
>     *   Destacar o playoff entre Los Angeles FC e América pela última vaga.
>     *   Analisar as chances dos clubes sul-americanos e europeus, criando expectativas para os confrontos.
> 
> Para aprofundar a pesquisa sobre o tema, posso usar as seguintes buscas no Google:
> 
> Com base nas informações coletadas, aqui está um plano de conteúdo detalhado sobre o Novo Mundial de Clubes da FIFA:
> 
> **Tópico:** Novo Mundial de Clubes da FIFA 2025
> 
> **Assuntos para Abordar no Post:**
> 
> 1.  **O que é o Novo Mundial de Clubes?**
>     *   Explicar o novo formato com 32 equipes, substituindo o antigo modelo.
>     *   Detalhar o período e o local de realização: de 14 de junho a 13 de julho de 2025, nos Estados Unidos.
>     *   Mencionar o objetivo da FIFA em tornar o torneio mais competitivo e global.
>     *   Formato do torneio: Fase de grupos com 8 grupos de 4 equipes, seguido de oitavas, quartas, semi e final em jogos únicos.
>     *   Não haverá disputa pelo terceiro lugar.
> 
> 2.  **Clubes Participantes e Expectativas**
>     *   Listar os clubes já classificados, incluindo os brasileiros: Palmeiras, Flamengo, Fluminense e Botafogo.
>     *   Destaque para outros clubes como Chelsea, Real Madrid, PSG, Bayern de Munique, Inter de Milão, Porto, Benfica e clubes de outras confederações.
>     *   Analisar os grupos e os possíveis confrontos, criando expectativas.
>     *   Grupos:
>         *   Grupo A: Palmeiras, Porto, Al Ahly, Inter Miami CF.
>         *   Grupo B: Botafogo, Paris Saint-Germain, Atlético de Madrid, Seattle Sounders FC.
>         *   Grupo D: Flamengo, Chelsea, Espérance de Tunis, e um clube a ser definido.
>         *   Grupo F: Fluminense, Borussia Dortmund, Ulsan HD, Mamelodi Sundowns.
>     *   Discutir as chances dos clubes sul-americanos e europeus.
>     *   Possível premiação de mais de 1 bilhão de dólares a serem distribuídos entre os clubes.
> 
> 3.  **Onde Assistir e Outros Detalhes**
>     *   Informar sobre os canais de transmissão, como a CazéTV, que transmitirá no mínimo 39 jogos.
>     *   Detalhes sobre as cidades-sede nos Estados Unidos e os estádios.
>     *   Citar o jogo de abertura entre Inter Miami e Al Ahly no Hard Rock Stadium.
>     *   A final será no MetLife Stadium em Nova Jersey.
> 
> **Hashtags:**
> 
> *   \#MundialDeClubes
> *   \#FIFA
> *   \#Futebol
> *   \#Clubes
> *   \#MundialDeClubesFIFA2025
> 
> Este plano oferece uma visão geral completa do Novo Mundial de Clubes da FIFA, ideal para gerar engajamento e informar o público sobre este grande evento esportivo.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ⚽️ O MUNDIAL DE CLUBES VAI TREMER AS AMÉRICAS! ⚽️
> 
> Preparem seus corações, fanáticos por futebol! O Novo Mundial de Clubes da FIFA está chegando com um formato INÉDITO e promete parar o mundo! 🌎
> 
> 🏆 O que mudou? Agora são 32 CLUBES disputando o título em um formato turbinado! De 14 de junho a 13 de julho de 2025, os Estados Unidos serão palco de um espetáculo que você não vai querer perder.
> 
> 🇧🇷 Clubes confirmados:
> Palmeiras no Grupo A, Botafogo no Grupo B, Flamengo no Grupo D e Fluminense no Grupo F.
> E tem mais: Chelsea, Real Madrid, PSG, Bayern de Munique, Inter de Milão, Porto, Benfica...
> E o jogão de abertura entre Inter Miami e Al Ahly no Hard Rock Stadium
> 
> 🔥 Emoção garantida: Fase de grupos ACIRRADA, seguida de oitavas, quartas, semi e a GRANDE FINAL! E não para por aí: premiação de mais de 1 bilhão de dólares a serem distribuídos entre os clubes.
> A final será no MetLife Stadium em Nova Jersey.
> 
> 📺 Onde assistir? Fique ligado! A CazéTV vai transmitir no mínimo 39 jogos! Prepare a torcida e não perca nenhum lance!
> 
> Quem leva essa? 👀 Deixe seu palpite nos comentários e compartilhe com a galera! 👇
> 
> \#MundialDeClubes \#FIFA \#Futebol \#Clubes \#MundialDeClubesFIFA2025
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais interessante para o público do Instagram, com foco em jovens de 18 a 30 anos. Aqui estão algumas sugestões:
> 
> *   **Tom:** O tom está um pouco formal. Podemos usar mais gírias e expressões que o público jovem usa.
> *   **Emojis:** Podemos adicionar mais emojis que representem os times e as seleções que vão participar.
> *   **Hashtags:** Podemos adicionar hashtags mais específicas para aumentar o alcance.
> 
> **Sugestão de post revisado:**
> 
> ⚽️ O MUNDIAL DE CLUBES VAI SER ÉPICO! ⚽️
> 
> Se preparem, apaixonados por futebol! O Novo Mundial de Clubes da FIFA tá chegando com um formato INÉDITO e vai ser SENSACIONAL! 🌎
> 
> 🏆 O que mudou? Agora são 32 CLUBES disputando o título em um formato turbinado! De 14 de junho a 13 de julho de 2025, os Estados Unidos vão tremer com esse espetáculo! 🏟️🇺🇸
> 
> 🇧🇷 Clubes confirmados:
> Palmeiras no Grupo A 🏆, Botafogo no Grupo B 🔥, Flamengo no Grupo D 🔴⚫ e Fluminense no Grupo F 💚.
> E tem mais: Chelsea 💙, Real Madrid 🤍, PSG ❤️💙, Bayern de Munique ❤️, Inter de Milão ⚫🔵, Porto 🔵⚪, Benfica 🔴...
> E o jogão de abertura vai ser entre Inter Miami 🩷 e Al Ahly 🇪🇬 no Hard Rock Stadium 🏟️
> 
> 🔥 Emoção garantida: Fase de grupos PEGADA, seguida de oitavas, quartas, semi e a GRANDE FINAL! E não para por aí: premiação de mais de 1 bilhão de dólares pra galera! 💰💰💰
> A final vai ser no MetLife Stadium em Nova Jersey 🏟️.
> 
> 📺 Onde assistir? Fiquem ligados! A CazéTV vai transmitir no mínimo 39 jogos! Preparem a torcida e não percam NADA! ⚽
> 
> Quem leva essa? 👀 Deixem seus palpites nos comentários e mandem pra geral! 👇
> 
> \#MundialDeClubes \#FIFA \#Futebol \#Clubes \#MundialDeClubesFIFA2025 \#CazéTV \#Palmeiras \#Botafogo \#Flamengo \#Fluminense \#InterMiami \#AlAhly
> 


--------------------------------------------------------------
